## Rough Alignment

In [ ]:
fp = FitPlaneElastic.from_points(histology_coords_px, oct_coords_mm, smoothing=smoothing)
rough_align_2d_image = fp.image_to_physical_z_projection(hist_img, x_range_mm, y_range_mm, 1e-3) # 1um/px
surface_z_um = fp.get_image_surface_z(x_range_mm, y_range_mm, pixel_size_mm) * 1000 # zmean_plane

plt.imshow(surface_z_um)
plt.colorbar()
plt.title("Surface Depth Estimated from Photobleach Barcodes")
plt.show()